In [1]:
import random

from distributed_processing.serializers import DummySerializer
from distributed_processing.client import Client
from distributed_processing.filesystem_connector import FileSystemConnector


In [2]:
CURRO = True

if CURRO:
    NS_PATH ="G:\\fs_namespaces\\prueba_distribuida"
    NS_PATH ="C:\\fs_namespaces\\prueba_distribuida"

else:
    NS_PATH = "/home/augusto/python/notebooks/fs_namespaces/prueba_distribuida"


In [3]:
import logging
logging.getLogger("distributed_processing").setLevel(logging.DEBUG)
#logging.getLogger("distributed_processing.filesystem_connector").setLevel(logging.DEBUG)

In [4]:
fs_connector = FileSystemConnector(NS_PATH)
fs_connector.with_watchdog=True
fs_connector.pop_watchdog_timeout = 10

client = Client(DummySerializer(), fs_connector, check_registry="cache")

registry_lock set
registry_lock unset
nclients_lock set


nclients_lock unset
Client with id: fs_client_4
Results queue: fs_client_4_responses


In [5]:
fs_connector.methods_registry()

registry_lock set
registry_lock unset


{'add': ['requests_cola_1'],
 'dic': ['requests_cola_1'],
 'div': ['requests_cola_1'],
 'eval_py_function': ['requests_py_eval'],
 'lista': ['requests_cola_1'],
 'mul': ['requests_cola_1'],
 'sleep': ['requests_cola_1'],
 'tupla': ['requests_cola_1']}

In [6]:
y = client.rpc_async("add", [1, 0])

Sent request with id: fs_client_4:1 to queue requests_cola_1


In [7]:
y.get()

1

In [8]:
client.check_registry ="always"

try:
    y = client.rpc_async("fake", [1, 0])
except Exception as e:
    print(e)

'method_queues_fake'


In [9]:
client.check_registry ="never" # use queue client.default_queue, by default "default"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

try:
    y.get()
except Exception as e:
    print(e)


Sent request with id: fs_client_4:2 to queue requests_cola_1


Error -32601 : The method does not exist/is not available.

 


In [10]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

y.safe_get(default="Esto es un error")

Sent request with id: fs_client_4:3 to queue requests_cola_1


'Esto es un error'

In [11]:
client.check_registry ="cache"

In [12]:
x = client.rpc_async("div", [1, 0])

Sent request with id: fs_client_4:4 to queue requests_cola_1


In [13]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "G:\python\distributed_processing\python\distributed_processing\worker.py", line 167, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agarcia\AppData\Local\Temp\ipykernel_24900\1531640066.py", line 8, in div
    return x / y
           ~~^~~
ZeroDivisionError: division by zero



In [14]:
client.rpc_sync("add", [1, 1])

Sent request with id: fs_client_4:5 to queue requests_cola_1


2

In [15]:
def f(x,y): return x + y

y = client.rpc_async_fn(f, [1, 2.0])

Sent request with id: fs_client_4:6 to queue requests_py_eval


In [16]:
y.get()

3.0

In [17]:
client.rpc_sync_fn(f, [3.0, 2.0])

Sent request with id: fs_client_4:7 to queue requests_py_eval


5.0

In [18]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "lista", "tupla", "dic"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

t = ("sleep", [10], {})
tp.append(t)
fs.append(client.rpc_async(t[0], t[1]))



Sent request with id: fs_client_4:8 to queue requests_cola_1
Sent request with id: fs_client_4:9 to queue requests_cola_1
Sent request with id: fs_client_4:10 to queue requests_cola_1
Sent request with id: fs_client_4:11 to queue requests_cola_1
Sent request with id: fs_client_4:12 to queue requests_cola_1
Sent request with id: fs_client_4:13 to queue requests_cola_1
Sent request with id: fs_client_4:14 to queue requests_cola_1
Sent request with id: fs_client_4:15 to queue requests_cola_1
Sent request with id: fs_client_4:16 to queue requests_cola_1


('add', [0.402695368850597, 0.8895395419335642], {})
('lista', [0.4072618751051509, 0.07709283265778166], {})
('dic', [0.8557629447951, 0.2635348478597973], {})
('mul', [0.7003913403445665, 0.3774671962726365], {})
('lista', [0.8173668596607947, 0.4091287348837197], {})
('div', [0.8599152331652826, 0.914408463861556], {})
('tupla', [0.6934656745983323, 0.580003823182854], {})
('mul', [0.09972881343035134, 0.01953440385013827], {})
('lista', [0.2806713992453813, 0.5560523891767433], {})


Sent request with id: fs_client_4:17 to queue requests_cola_1
Sent request with id: fs_client_4:18 to queue requests_cola_1


('mul', [0.9393637101022132, 0.8644570202479022], {})


In [19]:
resultados = [f.get() for f in fs]
resultados

[1.292234910784161,
 [0.4072618751051509, 0.07709283265778166],
 {'a': 0.8557629447951, 'b': [0.8557629447951, 0.2635348478597973]},
 0.26437475553349743,
 [0.8173668596607947, 0.4091287348837197],
 0.9404060298543738,
 (0.6934656745983323, 0.580003823182854),
 0.0019481429170435764,
 [0.2806713992453813, 0.5560523891767433],
 0.8120395537639734,
 None]

In [20]:
fs = client.rpc_multi_async(tp)

Sent request with id: fs_client_4:19 to queue requests_cola_1
Sent request with id: fs_client_4:20 to queue requests_cola_1
Sent request with id: fs_client_4:21 to queue requests_cola_1
Sent request with id: fs_client_4:22 to queue requests_cola_1
Sent request with id: fs_client_4:23 to queue requests_cola_1
Sent request with id: fs_client_4:24 to queue requests_cola_1
Sent request with id: fs_client_4:25 to queue requests_cola_1
Sent request with id: fs_client_4:26 to queue requests_cola_1
Sent request with id: fs_client_4:27 to queue requests_cola_1
Sent request with id: fs_client_4:28 to queue requests_cola_1
Sent request with id: fs_client_4:29 to queue requests_cola_1


In [21]:
# AsynResult.status updates the information every time it is called.
# 'PENDING' state should be assumed as transitory. 
# If there are responses available in the queue or in the cache 
# status or pending() updates the AsyncResult object.

[f.status for f in fs]

['OK',
 'OK',
 'OK',
 'OK',
 'OK',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING']

In [22]:
client.wait_responses(timeout=2)

['fs_client_4:29']

In [23]:
[f.status for f in fs]

['OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'PENDING']

In [24]:
client.wait_responses()

[]

In [25]:
# AsynResult.status updates information

[f.status for f in fs]

['OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK', 'OK']

In [26]:
try:
    client.wait_responses(["kk", "qq"])
except ValueError as e:
    print(e)

wait_responses: ['kk', 'qq'] neither in responses nor in pending.


In [27]:
client._update_cache_with_all_available_responses()

In [28]:
client.pending

{}

In [29]:
[f.get() for f in fs]

[1.292234910784161,
 [0.4072618751051509, 0.07709283265778166],
 {'a': 0.8557629447951, 'b': [0.8557629447951, 0.2635348478597973]},
 0.26437475553349743,
 [0.8173668596607947, 0.4091287348837197],
 0.9404060298543738,
 (0.6934656745983323, 0.580003823182854),
 0.0019481429170435764,
 [0.2806713992453813, 0.5560523891767433],
 0.8120395537639734,
 None]

In [30]:
fs = client.rpc_batch_async(tp)

Sent batch request with 11 requests to requests_cola_1


In [31]:
[f.get() for f in fs]

[1.292234910784161,
 [0.4072618751051509, 0.07709283265778166],
 {'a': 0.8557629447951, 'b': [0.8557629447951, 0.2635348478597973]},
 0.26437475553349743,
 [0.8173668596607947, 0.4091287348837197],
 0.9404060298543738,
 (0.6934656745983323, 0.580003823182854),
 0.0019481429170435764,
 [0.2806713992453813, 0.5560523891767433],
 0.8120395537639734,
 None]

In [32]:
client.rpc_batch_sync(tp)

Sent batch request with 11 requests to requests_cola_1


[1.292234910784161,
 [0.4072618751051509, 0.07709283265778166],
 {'a': 0.8557629447951, 'b': [0.8557629447951, 0.2635348478597973]},
 0.26437475553349743,
 [0.8173668596607947, 0.4091287348837197],
 0.9404060298543738,
 (0.6934656745983323, 0.580003823182854),
 0.0019481429170435764,
 [0.2806713992453813, 0.5560523891767433],
 0.8120395537639734,
 None]

In [33]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "fake"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)

('div', [0.2765083701932076, 0.2234683268424108], {})
('fake', [0.6301426853186024, 0.07146361929200529], {})
('fake', [0.2983557267097098, 0.9781838020861968], {})
('mul', [0.8490133031972757, 0.6536903788150596], {})
('mul', [0.0414764260475613, 0.822420030058196], {})
('add', [0.7119585548402398, 0.4903122448090771], {})
('add', [0.7486530169444501, 0.038544538995743616], {})
('mul', [0.7591366434769814, 0.435644970483661], {})
('div', [0.9240772779989194, 0.32377060073346275], {})
('div', [0.2879693268587079, 0.8596443292951291], {})


In [34]:
tp

[('div', [0.2765083701932076, 0.2234683268424108], {}),
 ('fake', [0.6301426853186024, 0.07146361929200529], {}),
 ('fake', [0.2983557267097098, 0.9781838020861968], {}),
 ('mul', [0.8490133031972757, 0.6536903788150596], {}),
 ('mul', [0.0414764260475613, 0.822420030058196], {}),
 ('add', [0.7119585548402398, 0.4903122448090771], {}),
 ('add', [0.7486530169444501, 0.038544538995743616], {}),
 ('mul', [0.7591366434769814, 0.435644970483661], {}),
 ('div', [0.9240772779989194, 0.32377060073346275], {}),
 ('div', [0.2879693268587079, 0.8596443292951291], {})]

In [35]:
client.check_registry ="never"
client.set_default_queue("cola_1")

fs = client.rpc_batch_async(tp)

Sent batch request with 10 requests to requests_cola_1


In [36]:
[f.safe_get() for f in fs]

[1.2373492660022487,
 None,
 None,
 0.5549918277860523,
 0.0341110435567419,
 1.202270799649317,
 0.7871975559401937,
 0.33071406064059505,
 2.8541111388913483,
 0.33498659508965783]

In [37]:
client.responses

{}

In [38]:
client.check_registry ="cache"

In [39]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [40]:
try:    
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [41]:
client.check_registry="never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

Sent request with id: fs_client_4:62 to queue requests_cola_1


In [42]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 


In [43]:
y = client.rpc_async("add", [1, 0])

Sent request with id: fs_client_4:63 to queue requests_cola_1


In [44]:
y.get(5)

1

In [45]:
def f(x,y): return x + y

client.check_registry = "never" # "never" use queue "default" don't work for rpc_async_fn or rpc_sync_fn 
y = client.rpc_async_fn(f, [1, 2.0])
try:
    print(y.get())
except Exception as e:
    print(e)

Sent request with id: fs_client_4:64 to queue requests_cola_1


Error -32601 : The method does not exist/is not available.

 


In [46]:
client.check_registry = "cache"
y = client.rpc_async_fn(f, [1, 2.0])

Sent request with id: fs_client_4:65 to queue requests_py_eval


In [47]:
y.get()

3.0

In [48]:
[k for k in client.responses.keys()]

[]

In [49]:
client.clean_used()

In [50]:
[k for k in client.responses.keys()]

[]

In [51]:
resultados

[1.292234910784161,
 [0.4072618751051509, 0.07709283265778166],
 {'a': 0.8557629447951, 'b': [0.8557629447951, 0.2635348478597973]},
 0.26437475553349743,
 [0.8173668596607947, 0.4091287348837197],
 0.9404060298543738,
 (0.6934656745983323, 0.580003823182854),
 0.0019481429170435764,
 [0.2806713992453813, 0.5560523891767433],
 0.8120395537639734,
 None]

In [52]:
client.rpc_sync_fn(print, ["hola"])

Sent request with id: fs_client_4:66 to queue requests_py_eval


In [61]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("sleep", "fake"))
    t = (fn, [10], {})
    print(t)
    tp.append(t)

('fake', [10], {})
('sleep', [10], {})
('sleep', [10], {})
('sleep', [10], {})
('fake', [10], {})
('sleep', [10], {})
('sleep', [10], {})
('fake', [10], {})
('fake', [10], {})
('sleep', [10], {})


In [101]:
client.check_registry="never"
fs = client.rpc_multi_async(tp)

Sent request with id: fs_client_4:157 to queue requests_cola_1
Sent request with id: fs_client_4:158 to queue requests_cola_1


Sent request with id: fs_client_4:159 to queue requests_cola_1
Sent request with id: fs_client_4:160 to queue requests_cola_1
Sent request with id: fs_client_4:161 to queue requests_cola_1
Sent request with id: fs_client_4:162 to queue requests_cola_1
Sent request with id: fs_client_4:163 to queue requests_cola_1
Sent request with id: fs_client_4:164 to queue requests_cola_1
Sent request with id: fs_client_4:165 to queue requests_cola_1
Sent request with id: fs_client_4:166 to queue requests_cola_1


In [102]:
from time import time

def gather(fs, tp, timeout=None, step=5):
    ids = [f.id for f in fs]
    t_0 = time()
    while time() - t_0 <= timeout:
        try:
            if fs[0]._client.wait_responses(ids, timeout=5) == []:
                return
        except TimeoutError:
            pass

        t = [((time() if f.finished_time is None else f.finished_time) -f.creation_time, f.status, f.id) for f in fs]
        pending = [x[0] for x in t if x[1]=="PENDING"]
        done = [x[0] for x in t if x[1] !="PENDING"]
        max_pending = max(pending) if len(pending)> 0 else 0 
        max_done = max(done) if len(done)>0 else 0
        print(max_done, max_pending)
        ids = [x[2] for x in t if x[1] =="PENDING"]


In [103]:
gather(fs, tp, 60, 5)

5.328825950622559 5.311243295669556
10.394101619720459 10.36086630821228
10.194912672042847 15.444279909133911
20.523454904556274 20.476256132125854
20.220745086669922 25.598377466201782
30.653213024139404 30.596716165542603
30.165385246276855 35.713247776031494
30.165385246276855 40.8109450340271
30.165385246276855 45.87255835533142
30.165385246276855 50.973963499069214
30.165385246276855 56.09268021583557
30.165385246276855 61.17133688926697


In [104]:
[f.status for f in fs]

['FAILED',
 'OK',
 'OK',
 'OK',
 'FAILED',
 'FAILED',
 'OK',
 'FAILED',
 'FAILED',
 'PENDING']

In [105]:
[(time() if f.finished_time is None else f.finished_time) -f.creation_time for f in fs]

[0.20798397064208984,
 10.194912672042847,
 10.185481548309326,
 20.184446096420288,
 10.15903925895691,
 20.167667865753174,
 30.165385246276855,
 20.155033588409424,
 20.220745086669922,
 61.206326484680176]

In [106]:
[f.finished_time for f in fs]

[1761063582.9501424,
 1761063592.9546537,
 1761063592.9624999,
 1761063602.9774225,
 1761063592.969486,
 1761063602.9930744,
 1761063613.0055604,
 1761063603.0171435,
 1761063603.0993521,
 None]